In [4]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class
class FallDetectionDataset(Dataset):
    def __init__(self, csv_files, labels, expected_frames=30, landmarks=18):
        self.csv_files = csv_files
        self.labels = labels
        self.expected_frames = expected_frames
        self.landmarks = landmarks

    def __len__(self):
        return len(self.csv_files)

    def __getitem__(self, idx):
        # Load CSV
        df = pd.read_csv(self.csv_files[idx])
        # Check number of frames
        num_frames = len(df) // self.landmarks
        if len(df) % self.landmarks != 0:
            print(f"Warning: CSV {self.csv_files[idx]} has {len(df)} rows, not divisible by {self.landmarks} landmarks.")
            # Truncate to nearest multiple of landmarks
            df = df.iloc[:num_frames * self.landmarks]
        
        # Extract x, y, z coordinates
        skeleton_data = df[['x', 'y', 'z']].values  # Shape: (num_frames * landmarks, 3)
        # Reshape to (num_frames, landmarks, 3)
        skeleton_data = skeleton_data.reshape(num_frames, self.landmarks, 3)

        # Pad if fewer than expected frames
        if num_frames < self.expected_frames:
            padding = np.zeros((self.expected_frames - num_frames, self.landmarks, 3))
            skeleton_data = np.vstack((skeleton_data, padding))
        elif num_frames > self.expected_frames:
            skeleton_data = skeleton_data[:self.expected_frames]

        # Flatten to 1D array (30 * 18 * 3 = 1620)
        skeleton_data = skeleton_data.flatten()  # Shape: (1620,)
        skeleton_tensor = torch.tensor(skeleton_data, dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return skeleton_tensor, label

# DNN Model Definition
class FallDetectionDNN(nn.Module):
    def __init__(self, input_size=1620, hidden_size=512, num_classes=2):
        super(FallDetectionDNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(hidden_size, hidden_size // 2)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.5)
        self.fc3 = nn.Linear(hidden_size // 2, num_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return x

# Training Function
def train_model(model, train_loader, criterion, optimizer, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

# Load and Prepare Data
def load_dataset(fall_dir, nofall_dir):
    fall_files = [os.path.join(fall_dir, f) for f in os.listdir(fall_dir) if f.endswith('.csv')]
    nofall_files = [os.path.join(nofall_dir, f) for f in os.listdir(nofall_dir) if f.endswith('.csv')]
    all_files = fall_files + nofall_files
    labels = [1] * len(fall_files) + [0] * len(nofall_files)
    return all_files, labels

# Main Training Script
if __name__ == "__main__":
    # Paths to CSV directories (adjust these)
    fall_dir = "processed/fall/csv"
    nofall_dir = "processed/nofall/csv"

    # Load data
    csv_files, labels = load_dataset(fall_dir, nofall_dir)

    # Filter CSVs with correct number of frames (optional)
    expected_frames = 30
    expected_rows = expected_frames * 18  # 18 landmarks
    filtered_files = []
    filtered_labels = []
    for csv_file, label in zip(csv_files, labels):
        df = pd.read_csv(csv_file)
        if len(df) >= expected_rows:
            filtered_files.append(csv_file)
            filtered_labels.append(label)
        else:
            print(f"Skipping {csv_file}: Only {len(df)} rows, expected {expected_rows}.")
    csv_files, labels = filtered_files, filtered_labels

    # Split into train and validation
    train_files, val_files, train_labels, val_labels = train_test_split(
        csv_files, labels, test_size=0.2, random_state=42
    )

    # Create datasets and dataloaders
    train_dataset = FallDetectionDataset(train_files, train_labels, expected_frames=30, landmarks=18)
    val_dataset = FallDetectionDataset(val_files, val_labels, expected_frames=30, landmarks=18)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

    # Initialize model, loss, and optimizer
    model = FallDetectionDNN(input_size=1620).to(device)  # 30 * 18 * 3 = 1620
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # Train the model
    print("Training DNN...")
    train_model(model, train_loader, criterion, optimizer, num_epochs=20)

    # Evaluate on validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    print(f"Validation Accuracy: {val_acc:.2f}%")

    # Save the model
    torch.save(model.state_dict(), "dnn_fall_detection.pth")
    print("Model saved as 'dnn_fall_detection.pth'")

Skipping processed/fall/csv/Coffee_room_01_video_44.csv: Only 414 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_02_video_67.csv: Only 126 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_02_video_60.csv: Only 450 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_41.csv: Only 450 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_02_video_61.csv: Only 0 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_30.csv: Only 486 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_24.csv: Only 504 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_22.csv: Only 522 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_13.csv: Only 486 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_6.csv: Only 522 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_29.csv: Only 522 rows, expected 540.
Skipping processed/fall/csv/Coffee_room_01_video_28.csv: 